In [155]:
import pandas as pd
import numpy as np

import math
import scipy.optimize as opt


In [156]:
# Data: Close
db = pd.read_excel('Data.xlsx', index_col = "Date")

# MSD

In [157]:
db.index = pd.to_datetime(db.index)
MSD_df = db['2006':'2013']
MSD_df = (MSD_df-MSD_df.mean())/MSD_df.std() #change rolling
MSD_df = MSD_df.dropna(axis = 1) #drop nan

In [158]:
target = {}
for i in range(0,len(MSD_df.columns)-1):
    for j in range(i+1,len(MSD_df.columns)):
        tmp = ((MSD_df.iloc[:,i] - MSD_df.iloc[:,j])**2).sum()
        target[str(MSD_df.columns[i])+','+str(MSD_df.columns[j])] = tmp

In [159]:
target_df = pd.DataFrame.from_dict(target, orient='index')
target_df = target_df.sort_values(by = 0)[0:10]

In [160]:
pair1_df_train = db.loc["2006":"2013",target_df.index[0].split(",")]
pair1_df_test = db.loc["2014":"2014",target_df.index[0].split(",")]
pair1_df_train

,DIS,TRV
Date,,
2006-01-03,24.070601,45.990002
2006-01-04,23.666136,46.500000
2006-01-05,24.080465,46.950001
2006-01-06,24.406010,47.209999
2006-01-09,24.662500,47.230000
...,...,...
2013-12-24,73.849998,89.489998
2013-12-26,74.620003,90.010002
2013-12-27,74.349998,89.849998


In [161]:
# ytlist = pair1_df_train.iloc[:,0] - pair1_df_train.iloc[:,1]

In [162]:
# from scipy.stats import t

# ytlist = [] #用來收集過去的yt們，算z_td、s_z時會用到

# #Table1 參數
# fi_1_0 = 0.1046
# fi_1_1 = 0.3886
# fi_2_0 = 0.0961
# fi_2_1 = -0.2403
# alpha_1_0 = 0.1639
# alpha_1_1 = 0.2132
# beta_1_1 = 0.6631
# alpha_2_0 = -0.1035
# alpha_2_1 = -0.1063
# beta_2_1 = -0.1751
# v = 7.1122
# gamma = 5.5563
# c_1 = -0.3569
# c_2 = 0.3136
# d = 1
# z_td = yt[len(yt)-d:-1] #t-d
# s_z = std(z_td)

# Fdis = (1+np.exp(-gamma*(z_td - c_1)(z_td - c_2))/s_z)**(-1)
# h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
# h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
# miu_1 = fi_1_0 + fi_1_1*y_t1
# miu_2 = fi_2_0 + fi_2_1*y_t1
# h_t = h_1_t + Fdis*h_2_t
# epsilon = t.rvs(v, size=1) #t-distribution
# a_t = h_t**0.5*epsilon #error term
# y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH


In [163]:
def fun_MLE(p):
     fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2,d, z_td, s_z = p
     # Define h_t1
     tmp = 30
     target = 1
     Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2))/s_z)**(-1)
     h_1_t = pair1_df_train.iloc[0:tmp,0].std()
     h_2_t = pair1_df_train.iloc[0:tmp,1].std()
     h_t1 = h_1_t + Fdis*h_2_t
     epsilon = t.rvs(v, size=1) #t-distribution
     a_t1 = h_t1**0.5*epsilon #error term


     y_t1 = pair1_df_train.iloc[tmp-1:tmp,0] - pair1_df_train.iloc[tmp-1:tmp,1]
     #Define sigma, real_y_t, y_t
     for i in range(tmp,len(pair1_df_train)):
          real_y_t = pair1_df_train.iloc[i:i+1,0] - pair1_df_train.iloc[i:i+1,1]
          real_y_t = real_y_t.to_numpy()
          h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
          h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
          miu_1 = fi_1_0 + fi_1_1*y_t1
          miu_2 = fi_2_0 + fi_2_1*y_t1
          h_t = h_1_t + Fdis*h_2_t
          epsilon = t.rvs(v, size=1) #t-distribution
          a_t = h_t**0.5*epsilon #error term
          y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH

          h_t1 = h_t
          a_t1 = a_t
          y_t1 = real_y_t

          sigma = h_t
          target *= np.sqrt(2*np.pi*sigma**2)*np.exp(-(real_y_t-y_t)**2/2*sigma**2) #MLE
     target = -target
     return target

In [164]:
#MLE

initial = np.array([0.1046, 0.3886, 0.0961, -0.2403, 0.1639, 0.2132, 0.6631, -0.1035, -0.1063, -0.1751, 7.1122, 5.5563, -0.3569, 0.3136, 1, 1, 1 ])  # 後面兩項問號
minimum = opt.fmin(func = fun_MLE, x0 = initial)
minimum[0]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Trade

In [ ]:
df_return = db.pct_change()

In [ ]:
global results

In [ ]:
results = {'signals':[],
           'returns':[],
           'A_SoldPrices':[],
           'A_BoughtPrices':[],
           'B_SoldPrices':[],
           'B_BoughtPrices':[]
           }

In [ ]:
# Input1 : Real return spread
# Input2 : Estimated return spread
# Output : Gain, Trading Signal

y1 = df_return.iloc[-1,0] 
y2 = df_return.iloc[-1,1]
p1 = db.iloc[-1:,0] #A Price
p2 = db.iloc[-1,1] #B Price

#First trade
i = 0

#signal
tmp0 = 0
tmp1 = 1
tmp2 = -1

#First trade
if i == 0:
    if (y1 > y2):
        signal = tmp1
        results['A_SoldPrices'].append(p1)
        results['B_BoughtPrices'].append(p2)
        i = 1
        tmp = 0
    
    elif (y1 < y2):  
        signal = tmp2
        results['A_BoughtPrices'].append(p1)
        results['B_SoldPrices'].append(p2) 
        i = 1
        tmp = 0
    else:
        signal = tmp0
        i = 0
        tmp = 1

elif i == 1:
    # After first trade
    if (y1 > y2) & (results['signals'][-1] == tmp1) :
        signal = tmp1
        results['A_SoldPrices'].append(y1)
        results['B_BoughtPrices'].append(y2)
        tmp = -np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])+np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1])
  
    elif (y1 <y2) & (results['signals'][-1] == tmp2):  
        signal = tmp2
        results['A_BoughtPrices'].append(y1)
        results['B_SoldPrices'].append(y2)
        tmp = np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])-np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1]) 

    else:
        signal = tmp0
        tmp = 0


results['signals'].append(signal)
results['returns'].append(tmp)